<a href="https://colab.research.google.com/github/LoolzMe/MachineLearning/blob/main/AIMLfC3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

import urllib

In [ ]:
(train_ds, test_ds, val_ds) = tfds.load('rock_paper_scissors', split=['train', 'test[70%:]', 'test[:30%]'], shuffle_files=True, as_supervised=True)

train_ds = train_ds.shuffle(1024).batch(2).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.shuffle(1024).batch(2).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(1024).batch(2).prefetch(tf.data.AUTOTUNE)

In [ ]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

weights_file = "inception_v3.h5"

urllib.request.urlretrieve(weights_url, weights_file)

('inception_v3.h5', <http.client.HTTPMessage at 0x7f42da11e310>)

In [ ]:
inceptionv3 = InceptionV3(input_shape=(300, 300, 3), include_top=False, weights=None)

inceptionv3.load_weights(weights_file)

In [ ]:
inceptionv3.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 149, 149, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 149, 149, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                      

In [ ]:
def freeze_model(model):
    for layer in model.layers:
        layer.trainable = False
    

In [ ]:
freeze_model(inceptionv3)

In [ ]:
IMG_SIZE = 300

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.2),
  layers.RandomTranslation(0.2, 0.2)
])

In [ ]:
big_model = Sequential([layers.InputLayer(input_shape=(300, 300, 3)), resize_and_rescale, data_augmentation])
big_model_inception = inceptionv3(big_model.output)

big_model = Model(big_model.input, big_model_inception)

output_layer = big_model.output

mini_model = layers.Flatten()(output_layer)
mini_model = layers.Dense(512, activation='selu')(mini_model)
mini_model = layers.Dropout(0.2)(mini_model)
mini_model = layers.Dense(256, activation='relu')(mini_model)
mini_model = layers.Dropout(0.2)(mini_model)
mini_model = layers.Dense(3, activation='softmax')(mini_model)

In [ ]:
model = Model(big_model.input, mini_model)

model.compile(optimizer='rmsprop', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model", save_best_only=True, save_freq=600, monitor='accuracy')

In [ ]:
with tf.device('/GPU:0'):
    model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
1260/1260 [==============================] - 71s 55ms/step - loss: 0.4322 - accuracy: 0.9683 - val_loss: 1.8518 - val_accuracy: 0.8571
Epoch 2/10
1260/1260 [==============================] - 71s 55ms/step - loss: 0.4893 - accuracy: 0.9690 - val_loss: 1.3433 - val_accuracy: 0.9018
Epoch 3/10
1260/1260 [==============================] - 70s 55ms/step - loss: 0.4350 - accuracy: 0.9710 - val_loss: 4.1736 - val_accuracy: 0.7143
Epoch 4/10
1260/1260 [==============================] - 71s 55ms/step - loss: 0.3000 - accuracy: 0.9766 - val_loss: 22.6138 - val_accuracy: 0.7054
Epoch 5/10
1260/1260 [==============================] - 71s 55ms/step - loss: 0.4505 - accuracy: 0.9778 - val_loss: 7.1976 - val_accuracy: 0.8214
Epoch 6/10
1260/1260 [==============================] - 71s 55ms/step - loss: 0.4434 - accuracy: 0.9690 - val_loss: 3.1074 - val_accuracy: 0.8750
Epoch 7/10
1260/1260 [==============================] - 71s 55ms/step - loss: 0.4005 - accuracy: 0.9738 - val_loss: 5.8040 

In [1]:
#only 20 epoches: 78% on test
model.evaluate(test_ds)

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -R './best_model' /content/gdrive/MyDrive/AIMfC3/